# Define your environment settings

In [ ]:
# Update the following values according to your lab environment!

# Host name of your IBM Cloud Pak for Data cluster
server = "cpd-cpd.apps.6777ccb323fb424f7dff4bfa.ocp.techzone.ibm.com"

# Generate an API authorization token as described in the following web page:
# https://www.ibm.com/docs/en/cloud-paks/cp-data/5.0.x?topic=apis-generating-api-auth-token
token = "YWRtaW46ekdybTI1MjQyRTUxNThuVkZ4UmhlZ29vbHpYaTVJNGFGM3FDNjNTYwo="

# Define a HTTP request header

In [ ]:
import http.client
import json

# First we define a header for the REST calls in this lab
# We use the same header for all REST calls in this notebook
auth_token = "ZenApiKey" + " " + token
header = {
    'Authorization': auth_token,
    'content-type': "application/json",
    'accept': "application/json"
    }
print( "Request Header:")
print( json.dumps(header, indent=4))

# Introduction to REST calls
## Execute a simple REST call of type "GET"

The first REST call that we will execute retrieves some configuration infos from the "Governance Console".\
In this example, we retrieve all tags defined in the "Governance Console".

You can also access these tags from the User Interface of the "Governance Console": 
* In the UI open the "Administration menu" (the "gear" icon in the upper right) and then select "Solution Configuration -> Tags"

In [ ]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/configuration/tags"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

## Execute a REST call with a parameter

Now we execute the same REST call as in the previous example. But we add a parameter "only_enabled"
This will retrieve only those tags that are currently enabled in the system.
You will notice that the result set contains less entries than before.
A question mark indicates the beginning of the parameters list.
Multiple parameters are separated by ampersand characters (see examples further down)

In [ ]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/configuration/tags?only_enabled=true"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

# Useful REST calls to interact with the Governance Console

In the next section, we introduce REST calls that are especially useful for **external systems**
to interact with the *Governance Console*.

Some examples:
* How to retrieve information about "Models" and "Use Cases" from the inventory
* How to retrieve the current status of a model
* How to create new "Models" and "Use Cases" in the inventory
* How to create new "Models" and "Use Cases" and pre-populate some of their fields with data
* How to link existing objects like a "Business Entity" to a model or use case
* How to query information about all currently running workflow instances
* How to query information about specific workflow instances, to check e.g. the current workflow stage

We use two useful methods to process the result set of a REST call:
* json.loads
* json.dumps

The json.loads() method is used to parse a valid JSON string and converts it into a Python Dictionary.
You can then easily access each data element in this dictionary.\
The json.dumps() method converts a Python object into a JSON formatted string.
We use a combination of both methods to pretty print the results of our REST calls.

## Query objects using SQL-like syntax

### Retrieve all Use Case descriptions from the inventory

In [ ]:
query = "SELECT+[Name],[Description]+FROM+[Register]" 
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/query?q="+query

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

### Retrieve all Model descriptions from the inventory

In [ ]:
query = "SELECT+[Title],[Name],[Description]+FROM+[Model]" 
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/query?q="+query

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

### Retrieve Model descriptions and iterate over result set

The next example shows in more detail how you can access the data returned by the REST call.\
We use method json.loads() to convert the result into a Python dictionary. Then we iterate over the rows of the result set.

A good way to understand the structure of a result set is to first print it out like this:
* print( json.dumps(json.loads(data), indent=4))

This will give you an idea about the general structure of the data.
Next you can generate a dictionary and access the data elements in this dictionary. For example:
* my_dict = json.loads(data)
* my_array = my_dict["rows"]

In [ ]:
# retrieve all "models" from the inventory

query = "SELECT+[Name],[Title]+FROM+[Model]" 
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/query?q="+query

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    #--------- Access returned data via a Python dictionary
    # Convert JSON string to Python dictionary
    my_dict = json.loads(data)
    print( "Definitions:\n", my_dict["definitions"])
    # Iterate over the rows in the result set of the query
    # and print the model names
    print( "\nRows:")
    my_array = my_dict["rows"]
    for x in my_array:
        model_name = x["fields"][0]["value"]
        print( model_name)
        title_dict = x["fields"][1]
        if "value" in title_dict:
             print( title_dict["value"])
    #--------- Format returned data into JSON string
    print( "\n\nResult in JSON format:\n")
    print( json.dumps(json.loads(data), indent=4))
    

## Retrieve objects by ID
The next REST call retrieves an object by its ID.\
To find a valid object ID you can access an object in the "Governance Console"
and then check the corresponding URL in the address bar of your web browser.\
The object ID can be found at the end of the URL.\
Here is an sample URL (the object ID in this example is 4033):\
https://cpd-cpd.apps.6777ccb323fb424f7dff4bfa.ocp.techzone.ibm.com/openpages-openpagesinstance-cr/app/jspview/react/grc/task-view/4033

In [ ]:
objectId = "4054"
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/contents/"+objectId

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

## Create new objects

The next REST call creates an object of type *Model* in the inventory of the *Governance console*.
It will also link this object to a parent objects of type *Business Entity*.
The ID of the parent object in this example is 4035. 
We also show below how to retrieve the ID of the new model.

After you have executed this REST call, you can go to the *Governance console*, 
open the main menu and select "Inventory -> Models" and search for "Customer_Segmentation".

**Note:**
Before you include references to other objects in a REST call, for example,
a reference to a parent object, you should first verify that those objects really exist.
This can be done via the *Governance console*. 
* For example, to verify that a specific business entity exists
you can open the "Governance console", then open the main menu and
select "Organisation -> Business Entities".
* To check if a certain user exists and to see the ID of the user, in the "Governance console"
you open the administration menu and select "Users and Security -> Users" and then click
on a user name. You will see the detailed user profile. The ID of the user can be found in the URL
of your web browser. For example, in the following URL the user's ID is "1002":\
https://cpd-cpd.apps.6777ccb323fb424f7dff4bfa.ocp.techzone.ibm.com/openpages-openpagesinstance-cr/app/jspview/react/grc/admin/users-edit/1002

### Create a new model

In [ ]:
endpoint= "/openpages-openpagesinstance-cr-opgrc/api/v2/contents"

request_type = "POST"
name = 'Customer_Segmentation'
payload = {
        'name': name,
        'description': name,
        'type_definition_id': '58',
        'primary_parent_id': '4035',
        'title': name
}
# Convert dictionary to JSON string
payload = json.dumps(payload)
# Execute REST call
conn = http.client.HTTPSConnection(server)
conn.request(request_type, endpoint, payload, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))

if response.status == 201:
    data = response.read().decode("utf-8")
    # Now we retrieve the ID of the new object from the REST call result
    my_dict = json.loads(data)
    object_id = my_dict["id"]
    print( "ID of the new object: {}\n\n".format( object_id))
    # Now we print the entire result in JSON format
    print( json.dumps(json.loads(data), indent=4))

### Create a new model and assign some field values

The next REST call again creates an object of type "Model".

But here we also define the value of the following additional fields:
* Field "Model or Non-Model" = "Model"
* Field "Machine Learning Model" = "Yes"

In [ ]:
endpoint= "/openpages-openpagesinstance-cr-opgrc/api/v2/contents"
request_type = "POST"
name = 'MyModel'
payload = {
        'name': name,
        'description': name,
        'type_definition_id': '58',
        'primary_parent_id': '4035',
        'title': name,
 'fields': [
      {
            'id': '1286',
            'data_type': 'ENUM_TYPE',
            'name': 'MRG-Model:Model Non Model',
             'value': {
                'id': '3696',
                'name': 'Model',
                'localized_label': 'Model',
                'index': 1,
             }
        },
	{
            'id': '1279',
            'data_type': 'ENUM_TYPE',
            'name': 'MRG-Model:Machine Learning Model',
             'value': {
                'id': '3694',
                'name': 'Yes',
                'localized_label': 'Yes',
                'index': 1,
              }
        }
]
}
# Convert dictionary to JSON string
payload = json.dumps(payload)

conn = http.client.HTTPSConnection(server)
conn.request(request_type, endpoint, payload, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))

if response.status == 201:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

### Create a "Use Case" or other object types
To create other types of objects, you need to specify their corresponding **type_definition_id** in the REST call.\
For example, a "use case" has type_definition_id 74. Hence you would use the following payload:
<pre>
payload = {
        'name': 'Some text',
        'description': 'Some text',
        'type_definition_id': '74',
        'primary_parent_id': 'Some ID',
        'title': 'Some text'
}
</pre>

Here are the type definition IDs of frequently used object types:
* Use Case: 74
* Model: 58
* Business Entity: 5
* File (e.g. document): 4
* ChangeRequest: 33
* Attestation: 24
* Issue: 6
* Action Item: 7

The following REST call retrieves all type definition IDs from the *Governance Console*.

In [ ]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/types"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

## Retrieve currently running workflows

Below are different types of queries to retrieve information about currently running workflow instances.

In [ ]:
# Retrieve workflow instances related to "Use Cases" (type "Register" is related to "Use Cases")
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?type_definition=Register"

# Retrieve workflow instances related to "Models"
#endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?type_definition=Model"

# Retrieve all instances related to a specific workflow , e.g. workflow ID = 161
#endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?workflow=161"

# Retrieve currently running workflows which are assigned to a specific user
#endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?user_role=assignee&user=admin"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))